In [1]:
import networkx as nx
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import math
import scipy
import matplotlib.pyplot as plt 
import random

In [2]:
bodyData = pd.read_csv("soc-redditHyperlinks-body.tsv", sep='\t')
titleData = pd.read_csv("soc-redditHyperlinks-title.tsv", sep='\t')
df = bodyData.append(titleData, ignore_index=True)
del df['PROPERTIES']

In [3]:
grouped_df =  df[['SOURCE_SUBREDDIT','TARGET_SUBREDDIT','LINK_SENTIMENT']]
df2 = grouped_df.groupby(
     ['SOURCE_SUBREDDIT','TARGET_SUBREDDIT']
 ).agg(
     SENTIMENT_SUM = ('LINK_SENTIMENT','sum')
 ).reset_index()

In [4]:
df2['SENTIMENT_SUM_ABS'] = abs(df2['SENTIMENT_SUM'])
df2 = df2.sort_values('SENTIMENT_SUM_ABS',ascending=False).reset_index(drop=True)

In [5]:
df2.head()

,SOURCE_SUBREDDIT,TARGET_SUBREDDIT,SENTIMENT_SUM,SENTIMENT_SUM_ABS
0,trendingsubreddits,changelog,548,548
1,moronicmondayandroid,android,478,478
2,goodshibe,dogecoin,299,299
3,nightlypick,hockey,296,296
4,titlegore,todayilearned,279,279


In [6]:
positive_sum = df2.loc[df2['SENTIMENT_SUM']>5]

In [7]:
positive_sum.head()

,SOURCE_SUBREDDIT,TARGET_SUBREDDIT,SENTIMENT_SUM,SENTIMENT_SUM_ABS
0,trendingsubreddits,changelog,548,548
1,moronicmondayandroid,android,478,478
2,goodshibe,dogecoin,299,299
3,nightlypick,hockey,296,296
4,titlegore,todayilearned,279,279


In [8]:
positive_sum.iloc[2700]

SOURCE_SUBREDDIT     howhumanbeingswork
TARGET_SUBREDDIT             psychology
SENTIMENT_SUM                        25
SENTIMENT_SUM_ABS                    25
Name: 2701, dtype: object

In [9]:
friend_list = []
for idx, row in positive_sum.iterrows():
    friend = (row['SOURCE_SUBREDDIT'],row['TARGET_SUBREDDIT'])
    friend_list.append(friend)

In [10]:
len(friend_list)

18781

In [16]:
negative_sum = df2.loc[df2['SENTIMENT_SUM']<-2]
negative_sum

,SOURCE_SUBREDDIT,TARGET_SUBREDDIT,SENTIMENT_SUM,SENTIMENT_SUM_ABS
2672,hearthstone,hearthstonecirclejerk,-25,25
3240,angryblackladies,blackladies,-22,22
5656,drama,news,-15,15
6522,publichealthwatch,abortion,-13,13
6661,drama,videos,-13,13
...,...,...,...,...
45398,atheism,debateachristian,-3,3
45446,publichealthwatch,rapecounseling,-3,3
45464,publichealthwatch,rape,-3,3
45781,elitistwatch,metaljerk,-3,3


In [12]:
enemy_list = []
for idx, row in negative_sum.iterrows():
    enemy = (row['SOURCE_SUBREDDIT'],row['TARGET_SUBREDDIT'])
    enemy_list.append(enemy)

In [13]:
len(enemy_list)

284

In [14]:
def friend_enemy_list(df):
    grouped_df =  df[['SOURCE_SUBREDDIT','TARGET_SUBREDDIT','LINK_SENTIMENT']]
    df2 = grouped_df.groupby(
         ['SOURCE_SUBREDDIT','TARGET_SUBREDDIT']
     ).agg(
         SENTIMENT_SUM = ('LINK_SENTIMENT','sum')
     ).reset_index()
    df2['SENTIMENT_SUM_ABS'] = abs(df2['SENTIMENT_SUM'])
    df2 = df2.sort_values('SENTIMENT_SUM_ABS',ascending=False).reset_index(drop=True)
    positive_sum = df2.loc[df2['SENTIMENT_SUM']>5]
    friend_list = []
    for idx, row in positive_sum.iterrows():
        friend = (row['SOURCE_SUBREDDIT'],row['TARGET_SUBREDDIT'])
        friend_list.append(friend)
    negative_sum = df2.loc[df2['SENTIMENT_SUM']<-2]
    
    enemy_list = []
    for idx, row in negative_sum.iterrows():
        enemy = (row['SOURCE_SUBREDDIT'],row['TARGET_SUBREDDIT'])
        enemy_list.append(enemy)
        
    return friend_list, enemy_list

In [37]:
popular_friends_source = positive_sum.groupby(['SOURCE_SUBREDDIT']).agg(
     Friend_count = ('TARGET_SUBREDDIT','count')
 ).reset_index()
popular_friends_source.sort_values('Friend_count',ascending=False).head(10)

,SOURCE_SUBREDDIT,Friend_count
572,bestof,473
5184,subredditdrama,395
5536,titlegore,292
4829,shitredditsays,160
5262,switcharoo,159
4801,shitamericanssay,140
4823,shitpost,117
2367,hailcorporate,103
993,circlebroke2,91
4816,shitliberalssay,85


In [29]:
popular_friends_target = positive_sum.groupby(['TARGET_SUBREDDIT']).agg(
     Friend_count = ('TARGET_SUBREDDIT','count')
 ).reset_index()
popular_friends_target.sort_values('Friend_count',ascending=False).head(10)

,TARGET_SUBREDDIT,Friend_count
249,askreddit,641
1822,iama,424
2846,pics,274
3823,todayilearned,225
4016,videos,222
1484,funny,221
4159,worldnews,213
4183,writingprompts,161
1525,gaming,147
2077,leagueoflegends,145


In [34]:
popular_enemies_source= negative_sum.groupby(['SOURCE_SUBREDDIT']).agg(
     Enemy_count = ('SOURCE_SUBREDDIT','count')
 ).reset_index()
popular_enemies_source.sort_values('Enemy_count',ascending=False).head(10)

,SOURCE_SUBREDDIT,Enemy_count
32,drama,32
109,subredditdrama,29
107,subreddit_stats,12
81,publichealthwatch,12
101,sjwnews,10
21,botsrights,9
3,againstmensrights,7
25,circlebroke2,6
31,deadpoolhatersclub,5
39,evenwithcontext,5


In [35]:
popular_enemies_target = negative_sum.groupby(['TARGET_SUBREDDIT']).agg(
     Enemy_count = ('TARGET_SUBREDDIT','count')
 ).reset_index()
popular_enemies_target.sort_values('Enemy_count',ascending=False).head(10)

,TARGET_SUBREDDIT,Enemy_count
118,news,12
69,funny,8
201,videos,6
16,askreddit,6
58,enoughtrumpspam,6
184,todayilearned,5
53,drama,4
134,pics,4
6,againstmensrights,3
70,gamerghazi,3
